In [ ]:
!pip install -U efficientnet

In [1]:
# Importing modules
import numpy as np 
import pandas as pd 
import os
from keras.preprocessing.image import load_img, img_to_array
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.metrics import AUC
from keras.layers import *
import efficientnet.keras as efn
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import StratifiedShuffleSplit

Using TensorFlow backend.


In [2]:
# Loading in the data
X_train = np.load("../input/siimisic-melanoma-resized-images/x_train_224.npy")
y_train = pd.read_csv("../input/siim-isic-melanoma-classification/train.csv")['target'].values

In [3]:
# Getting cross validation splits
sss = StratifiedShuffleSplit(n_splits=5, test_size=.1, random_state=1)
train_ind, cv_ind = [], []
for train_index, cv_index in sss.split(X_train, y_train):
    train_ind.append(train_index)
    cv_ind.append(cv_index)

In [7]:
# Loading in efn4
efn4 = efn.EfficientNetB4(input_shape=(224, 224, 3), weights="imagenet", include_top=False)

In [14]:
# Getting architecture of model function
def get_model_compile():
    model = keras.Sequential()
    model.add(efn4)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1024, activation="relu"))
    model.add(Dropout(.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=[AUC(name='auc')])
    return model

In [ ]:
# Fitting models
model0 = get_model()
model0.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=[AUC(name='auc')])
mcp_save = ModelCheckpoint('model_weights0.hdf5', save_best_only=True, monitor='val_auc', mode='max')
model0.fit(X_train[train_ind[0]], y_train[train_ind[0]],
          validation_data=(X_train[cv_ind[0]], y_train[cv_ind[0]]),
          epochs=3, callbacks=[mcp_save], verbose=2)

In [ ]:
# Fitting models
model1 = get_model()
model1.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=[AUC(name='auc')])
mcp_save = ModelCheckpoint('model_weights1.hdf5', save_best_only=True, monitor='val_auc', mode='max')
model1.fit(X_train[train_ind[1]], y_train[train_ind[1]],
          validation_data=(X_train[cv_ind[1]], y_train[cv_ind[1]]),
          epochs=4, callbacks=[mcp_save], verbose=2)

In [ ]:
# Fitting models
model2 = get_model()
model2.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=[AUC(name='auc')])
mcp_save = ModelCheckpoint('model_weights2.hdf5', save_best_only=True, monitor='val_auc', mode='max')
model2.fit(X_train[train_ind[2]], y_train[train_ind[2]],
          validation_data=(X_train[cv_ind[2]], y_train[cv_ind[2]]),
          epochs=4, callbacks=[mcp_save], verbose=2)

In [ ]:
# Fitting models
model3 = get_model()
model3.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=[AUC(name='auc')])
mcp_save = ModelCheckpoint('model_weights3.hdf5', save_best_only=True, monitor='val_auc', mode='max')
model3.fit(X_train[train_ind[3]], y_train[train_ind[3]],
          validation_data=(X_train[cv_ind[3]], y_train[cv_ind[3]]),
          epochs=4, callbacks=[mcp_save], verbose=2)

In [8]:
# Fitting models
model4 = get_model()
model4.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=[AUC(name='auc')])
mcp_save = ModelCheckpoint('model_weights4.hdf5', save_best_only=True, monitor='val_auc', mode='max')
model4.fit(X_train[train_ind[4]], y_train[train_ind[4]],
          validation_data=(X_train[cv_ind[4]], y_train[cv_ind[4]]),
          epochs=4, callbacks=[mcp_save], verbose=2)

Train on 29813 samples, validate on 3313 samples
Epoch 1/4
 - 801s - loss: 0.0925 - auc: 0.7395 - val_loss: 0.0743 - val_auc: 0.8283
Epoch 2/4
 - 719s - loss: 0.0613 - auc: 0.8952 - val_loss: 0.0779 - val_auc: 0.8041
Epoch 3/4
 - 718s - loss: 0.0439 - auc: 0.9542 - val_loss: 0.0910 - val_auc: 0.8279
Epoch 4/4
 - 718s - loss: 0.0262 - auc: 0.9792 - val_loss: 0.1104 - val_auc: 0.7366


In [9]:
del X_train, y_train

In [12]:
# Loading in test data and image names
test_img_names = pd.read_csv("../input/siim-isic-melanoma-classification/test.csv")['image_name'].values
x_test = np.load("../input/siimisic-melanoma-resized-images/x_test_224.npy")

In [15]:
# Getting models to load in best weights
model0, model1, model2, model3, model4 = [get_model_compile() for num in range(5)]

In [16]:
# Loading best weights for each model and making predictions
model0.load_weights("model_weights0.hdf5")
model1.load_weights("model_weights1.hdf5")
model2.load_weights("model_weights2.hdf5")
model3.load_weights("model_weights3.hdf5")
model4.load_weights("model_weights4.hdf5")
predictions0 = model0.predict(x_test).flatten()
predictions1 = model1.predict(x_test).flatten()
predictions2 = model2.predict(x_test).flatten()
predictions3 = model3.predict(x_test).flatten()
predictions4 = model4.predict(x_test).flatten()

In [17]:
# Combining results
final_predictions = (predictions0 + predictions1 + predictions2 +
                    predictions3 + predictions4) / 5

# Submission dataset
final_sub = pd.DataFrame({"image_name": test_img_names, "target": final_predictions})
final_sub.to_csv('efn_cv.csv', index=False)